### 把cosmopedia转为输入形式

In [20]:
import pandas as pd
from pathlib import Path
import re

def print_in_lines(text, line_length):
    words        = text.split()
    lines        = []
    current_line = ''
    for word in words:
        if len(current_line + word) <= line_length:
            current_line = current_line + word + ' '
        else:
            lines.append(current_line.strip())
            current_line = word + ' '
    lines.append(current_line.strip())
    for line in lines:
        print(line)

dataset_dir = Path(r'H:\2024_03_05课程大纲\数据\syllabus\datasets\cosmopedia')
data = pd.read_parquet(dataset_dir / 'khanacademy' / 'train-00000-of-00001.parquet')

In [2]:
data = pd.read_parquet(dataset_dir / 'khanacademy' / 'train-00000-of-00001.parquet')

In [6]:
data.audience

,prompt,text_token_length,text,seed_data,format,audience
0,Write a long and very detailed course unit for...,975,Lesson Objective: Students will understand ho...,khanacademy,textbook_narrative_tone,general
1,Write a long and very detailed course unit for...,825,"In this sub-unit, we will explore counting in...",khanacademy,textbook_narrative_tone,general
2,Write a long and very detailed course unit for...,746,A number grid is a simple yet powerful tool f...,khanacademy,textbook_narrative_tone,general
3,Write a long and very detailed course unit for...,1197,"In this sub-unit, we will explore the concept...",khanacademy,textbook_narrative_tone,general
4,Write a long and very detailed course unit for...,676,"Let's dive into our next lesson, ""Counting by...",khanacademy,textbook_narrative_tone,general
...,...,...,...,...,...,...
24118,Write a long and very detailed course unit for...,706,"Bare/Bear:\n\nThe word ""bare"" means exposed o...",khanacademy,textbook_narrative_tone,general
24119,Write a long and very detailed course unit for...,972,"The Sound of Language: Alliteration, Assonanc...",khanacademy,textbook_narrative_tone,general
24120,Write a long and very detailed course unit for...,936,"In this lesson, we will explore the concept o...",khanacademy,textbook_narrative_tone,general
24121,Write a long and very detailed course unit for...,640,Understatement is a stylistic device used to ...,khanacademy,textbook_narrative_tone,general


In [47]:
re.search('Write the new sub-unit titled "(.*?)" while trying to be:', item['prompt'])

<re.Match object; span=(440, 497), match='Write the new sub-unit titled "Irony" while tryin>

In [6]:
# 小学水平 khanacademy

# 大学水平openstax 和stanford
khanacademy_prompt_tempalte = {
    'textbook_name': ['Write a long and very detailed course unit for a textbook on "(.*?) - .*"'],
    'chapter_name': ['Write a long and very detailed course unit for a textbook on ".* - (.*?)"'],
    'unit_name': ['We are currently writing the first chapter: "Unit \d+: .* - (.*?)"', 'and are now writing a chapter on "Unit \d+: .* - (.*?)"'],
    'lesson_name': ['Write the new sub-unit titled "(.*?)" while trying to be:'],
}

# # Item格式，弃用
# {
#     'textbook_name': 'Early math review',
#     'chapters': [
#         {
#             'chapter_name': '',
#             'units': [
#                 {
#                     'unit_name': '',
#                     'lessons': [
#                         'lesson_name': '',
#                         'text': '',
#                     ]
#                 }
#             ]
#         }
#     ]
# }

# {
#     'textbook_name': '',
#     'paragraphs' = [{
#         'text', # 必有
#         'position', # 必有
#         'lesson_name', 
#     }],
# }

import json, tqdm
output_path = dataset_dir / 'khanacademy/inputs'

textbook_name = ''

# current_dict = {'textbook_name': '', 'paragraphs': []}

for index, row in tqdm.tqdm(data.iterrows()):
    item = row.to_dict()

    name_mapper = {}
    for name, templates in khanacademy_prompt_tempalte.items():
        for template in templates:
            if res := re.search(template, item['prompt']):
                value = res.group(1)
                name_mapper[name] = value
                break
        else:
            raise Exception(f'{index} Not Match for template `{name}`')
    
    if textbook_name != name_mapper['textbook_name']:
        textbook_name = name_mapper['textbook_name'].replace('/', '')

    output_file = (output_path / textbook_name).open('a', encoding='utf-8')
    output_file.write(json.dumps({
        'text': item['text'],
        'position': f'{name_mapper["chapter_name"]}_{name_mapper["unit_name"]}_{name_mapper["lesson_name"]}',
        'token_length': item['text_token_length']
    }) + '\n')
    output_file.close()

24123it [00:07, 3032.53it/s]


In [4]:
    output_file.close()


In [58]:
data.prompt

,text_token_length,prompt,text,seed_data,format,audience
0,1203,Write a comprehensive and in-depth textbook on...,**Global Stratification**\n\n*Global stratifi...,openstax,textbook_academic,college_studnets
1,1174,Write a comprehensive and in-depth textbook on...,**Unit: Calculating Keynesian Policy Interven...,openstax,textbook_academic,college_studnets
2,590,Create a fun and simple e-learning module on '...,Title: A Fun Adventure with Jobs the Turtle!\...,openstax,e-learning_module,young_children
3,1422,Write a comprehensive and in-depth textbook on...,**8. Operations and Management Plan**\n\nIn t...,openstax,textbook_academic,college_studnets
4,489,Write a comprehensive and in-depth textbook on...,III. Application (10-25 min)\n\nA. Analyzing ...,openstax,textbook_academic,college_studnets
...,...,...,...,...,...,...
63161,445,Create a fun and simple e-learning module on '...,Welcome to our exciting e-learning module on ...,openstax,e-learning_module,young_children
63162,683,Create a fun and simple e-learning module on '...,Welcome to our exciting adventure in Algebra ...,openstax,e-learning_module,young_children
63163,529,Create a fun and simple e-learning module on '...,Welcome to our e-learning adventure! Today we...,openstax,e-learning_module,young_children
63164,667,Create an extract of a scientific journal arti...,Title: Bacterial Conjunctivitis: An Explorati...,openstax,scientific_article,researchers


In [61]:
print_in_lines(data.prompt[0], 100)

Write a comprehensive and in-depth textbook on 'Introduction to Sociology 3e', tailored for college
students. We are writing on chapter 'Global Inequality'. In particular, section 'Global
Stratification and Classification'. Write a new unit titled 'Global Stratification'. Here's some
text for inspiration: While stratification in the United States refers to the unequal distribution
of resources among individuals, global stratification refers to this unequal distribution among
nations. There are two dimensions to this stratification: gaps between nations and gaps within
nations. When it comes to global inequality, both economic inequality and social inequality may
concentrate the burden of poverty among certain segments of the earth’s population (Myrdal 1970).
Try to be: - Rigorous: Ensure very detailed and in-depth coverage of the concepts. - Engaging: Write
with an academic and engaging tone that captivates interest. - Applied: Use specific and practical
examples. For example, if the t

In [32]:
for index, prompt in enumerate(data.prompt):
    # print_in_lines(prompt, 100)
    print(prompt)
    print('----------')

    if index > 150:
        break

Write a long and very detailed course unit for a textbook on "Early math review - Counting".
We are currently writing the first chapter: "Unit 1: Counting - Counting".
You will be writing the first lesson for this chapter.
Write the new sub-unit titled "Counting with small numbers" while trying to be:
- Rigorous - you create challenging textbooks that cover the material in depth.
- Engaging - your textbooks have a narrative arc and engaging tone, like the writing of Michael Lewis.
- Applied - you use specific and practical examples. For example, if the topic is integration in calculus, include equations and proofs of the concept you're teaching. As another example, if the topic is the history of the United States, include dates, names, and key events.
Do not include a title or an introduction, simply write the content without headlines and introductory phrases. Focus purely on the subject itself without offering advice on teaching and instruction.
----------
Write a long and very detai

In [24]:
print_in_lines(data.prompt[0], 100)

Write a long and very detailed course unit for a textbook on "Early math review - Counting". We are
currently writing the first chapter: "Unit 1: Counting - Counting". You will be writing the first
lesson for this chapter. Write the new sub-unit titled "Counting with small numbers" while trying to
be: - Rigorous - you create challenging textbooks that cover the material in depth. - Engaging -
your textbooks have a narrative arc and engaging tone, like the writing of Michael Lewis. - Applied
- you use specific and practical examples. For example, if the topic is integration in calculus,
include equations and proofs of the concept you're teaching. As another example, if the topic is the
history of the United States, include dates, names, and key events. Do not include a title or an
introduction, simply write the content without headlines and introductory phrases. Focus purely on
the subject itself without offering advice on teaching and instruction.


In [25]:
print_in_lines(data.prompt[2], 100)

Write a long and very detailed course unit for a textbook on "Early math review - Counting". We have
already covered chapter(s) "1. Counting" and are now writing a chapter on "Unit 1: Counting -
Numbers 0 to 120". You will be writing the first lesson for this chapter. Write the new sub-unit
titled "Number grid" while trying to be: - Rigorous - you create challenging textbooks that cover
the material in depth. - Engaging - your textbooks have a narrative arc and engaging tone, like the
writing of Michael Lewis. - Applied - you use specific and practical examples. For example, if the
topic is integration in calculus, include equations and proofs of the concept you're teaching. As
another example, if the topic is the history of the United States, include dates, names, and key
events. Do not include a title or an introduction, simply write the content without headlines and
introductory phrases. Focus purely on the subject itself without offering advice on teaching and
instruction.


### 把bigsurvey转为输入形式

In [12]:
bigsurvey_dir = dataset_dir.parent / 'bigsurvey/MDS_truncated'

source_file, target_file = bigsurvey_dir / 'test_truncate.src.txt', bigsurvey_dir / 'test_truncate.tgt.txt'

for index, (source, target) in enumerate(zip(source_file.read_text().split('\n'), target_file.read_text().split('\n')), 1):
    output_file = bigsurvey_dir / 'inputs' / f'test_{index}'
    target_file = bigsurvey_dir / 'summary' / f'test_{index}'
    paras = [json.dumps({
        'text': text,
        'position': 'x'
    }) for text in source.split('story_separator_special_tag')]    

    output_file.write_text('\n'.join(paras))
    target_file.write_text(target)

## OpenStax

In [15]:
import pandas as pd
from pathlib import Path
import re

def print_in_lines(text, line_length):
    words        = text.split()
    lines        = []
    current_line = ''
    for word in words:
        if len(current_line + word) <= line_length:
            current_line = current_line + word + ' '
        else:
            lines.append(current_line.strip())
            current_line = word + ' '
    lines.append(current_line.strip())
    for line in lines:
        print(line)

dataset_dir = Path(r'H:\2024_03_05课程大纲\数据\syllabus\datasets\cosmopedia')
data = pd.read_parquet(dataset_dir / 'openstax' / 'train-00000-of-00002.parquet')

In [16]:
data.head(10)

,text_token_length,prompt,text,seed_data,format,audience
0,1203,Write a comprehensive and in-depth textbook on...,**Global Stratification**\n\n*Global stratifi...,openstax,textbook_academic,college_studnets
1,1174,Write a comprehensive and in-depth textbook on...,**Unit: Calculating Keynesian Policy Interven...,openstax,textbook_academic,college_studnets
2,590,Create a fun and simple e-learning module on '...,Title: A Fun Adventure with Jobs the Turtle!\...,openstax,e-learning_module,young_children
3,1422,Write a comprehensive and in-depth textbook on...,**8. Operations and Management Plan**\n\nIn t...,openstax,textbook_academic,college_studnets
4,489,Write a comprehensive and in-depth textbook on...,III. Application (10-25 min)\n\nA. Analyzing ...,openstax,textbook_academic,college_studnets
5,1104,Create an extract of a scientific journal arti...,**The Bohr Atom: A Revolutionary Model Reeval...,openstax,scientific_article,researchers
6,730,Create an engaging and accessible e-learning m...,"Once upon a time, there was a young inventor ...",openstax,e-learning_module,middle_school_students
7,833,Create an extract of a scientific journal arti...,"Introduction\n\nIn this unit, we delve into t...",openstax,scientific_article,researchers
8,974,Create an extract of a scientific journal arti...,**Unit Title:** Oxygenic and Anoxygenic Photo...,openstax,scientific_article,researchers
9,525,Create a fun and simple e-learning module on '...,"Welcome to ""Nature vs. Nurture: The Great Squ...",openstax,e-learning_module,young_children


In [18]:
for prompt in data.prompt[:100]:
    print_in_lines(prompt, 100)

    print('------')

Write a comprehensive and in-depth textbook on 'Introduction to Sociology 3e', tailored for college
students. We are writing on chapter 'Global Inequality'. In particular, section 'Global
Stratification and Classification'. Write a new unit titled 'Global Stratification'. Here's some
text for inspiration: While stratification in the United States refers to the unequal distribution
of resources among individuals, global stratification refers to this unequal distribution among
nations. There are two dimensions to this stratification: gaps between nations and gaps within
nations. When it comes to global inequality, both economic inequality and social inequality may
concentrate the burden of poverty among certain segments of the earth’s population (Myrdal 1970).
Try to be: - Rigorous: Ensure very detailed and in-depth coverage of the concepts. - Engaging: Write
with an academic and engaging tone that captivates interest. - Applied: Use specific and practical
examples. For example, if the t

In [12]:
data.audience.unique()

array(['researchers', 'young_children', 'college_students',
       'high_school_studnets'], dtype=object)

### Stanford

In [1]:
import pandas as pd
from pathlib import Path
import re

def print_in_lines(text, line_length):
    words        = text.split()
    lines        = []
    current_line = ''
    for word in words:
        if len(current_line + word) <= line_length:
            current_line = current_line + word + ' '
        else:
            lines.append(current_line.strip())
            current_line = word + ' '
    lines.append(current_line.strip())
    for line in lines:
        print(line)

dataset_dir = Path(r'H:\2024_03_05课程大纲\数据\syllabus\datasets\cosmopedia')
data = pd.read_parquet(dataset_dir / 'stanford' / 'train-00000-of-00013.parquet')

In [6]:
print_in_lines(data.prompt[1], 100)

Write a long and very detailed course unit for a textbook on "Fourth-Year Japanese, Second Quarter"
intended for young children. We have already covered chapter(s) "1. Introduction", "2. Structure of
Japanese", "3. Writings in different genres and styles", "4. Using knowledge in writing", "5.
Expressing opinions on a variety of topics", "6. Original writings" and are now writing a chapter on
"7. Conclusion". We have already covered the following sub-units in the current chapter: "7.1.
Review of course content". Write the new sub-unit titled "7.2. Future directions for Japanese
language learning" while trying to be: - Rigorous - you create challenging textbooks that cover the
material in depth. - Engaging - your textbooks have a narrative arc and engaging tone, like the
writing of Michael Lewis. - Applied - you use specific and practical examples. For example, if the
topic is integration in calculus, include equations and proofs of the concept you're teaching. As
another example, if the

In [4]:
print_in_lines(data.prompt[0], 100)

Write a long and very detailed course unit for a textbook on "10 French Women Who Changed History"
intended for professionals and researchers in the field. We are currently writing the first chapter:
"1. Introduction". You will be writing the first sub-unit for this chapter. Write the new sub-unit
titled "1.1. Overview of the course" while trying to be: - Rigorous - you create challenging
textbooks that cover the material in depth. - Engaging - your textbooks have a narrative arc and
engaging tone, like the writing of Michael Lewis. - Applied - you use specific and practical
examples. For example, if the topic is integration in calculus, include equations and proofs of the
concept you're teaching. As another example, if the topic is the history of the United States,
include dates, names, and key events. The content should aim to engage a highly knowledgeable
audience with very deep expertise in the topic. Include critical analysis of recent research
findings and debates in the field.


In [5]:
print_in_lines(data.text[0], 100)

1.1 Overview of the Course In this comprehensive textbook, we delve into the lives and achievements
of ten influential French women who left indelible marks on history. Our focus extends beyond mere
biographical sketches; instead, we examine each woman's contributions within their respective fields
and assess how these advancements reverberate even today. Through rigorous examination and engaging
storytelling, our goal is not only to provide an authoritative account but also to spark curiosity
among seasoned scholars about fresh perspectives and ongoing scholarly debates. Our selection
encompasses various domains such as politics, science, literature, art, activism, and philosophy –
all areas where these trailblazing figures made significant strides despite facing societal
constraints imposed by gender roles and expectations prevalent during their times. They navigated
through challenges with resilience and determination, ultimately transforming society and leaving
behind legacies wort

### 把Khanacademy输出归一化一下

In [11]:
from pathlib import Path; import re; import tqdm
not_tab = 0
dataset_dir = Path(r'H:\2024_03_05课程大纲\数据\syllabus\datasets\cosmopedia')
for file in tqdm.tqdm((dataset_dir / 'khanacademy/sentences_bak').glob('*')):
    sentences = []
    sentence = ''
    for line in file.read_text(encoding='utf-8').split('\n'):
        if not line: continue
        if re.search('\d+-\d+$', line):
            sentence += line
            if len(sentence.split('\t')) != 2:
                *lst, position = sentence.split('\t')
                sentence = ' '.join(lst) + '\t' + position
            sentences.append(sentence)
            sentence = ''
        else:
            sentence += line
    (dataset_dir / 'khanacademy/sentences' / file.name).write_text('\n'.join(sentences))

130it [00:16,  7.73it/s]


### RSM和foundations of database进行输入归一化


In [5]:
import tqdm; from pathlib import Path
import nltk

rsm_dir = Path(r'H:\2024_03_05课程大纲\数据\syllabus\datasets\rsm\rsm_coref')
output_path = Path(r'H:\2024_03_05课程大纲\数据\syllabus\datasets\rsm\sentences\rsm')

sentences = []

for path in tqdm.tqdm(rsm_dir.glob('*'), desc='loading rsm'):
    for para_index, paragraph in enumerate(path.read_text(encoding='utf-8').split('\n'), 1):
        for sent_index, sentence_text in enumerate(nltk.tokenize.sent_tokenize(paragraph), 1):
            sentence_text = sentence_text.replace('\t', ' ')
            position = f"{path.name.split(' ')[0]}-{para_index}-{sent_index}"
            sentences.append(f'{sentence_text}\t{position}')
    
output_path.write_text('\n'.join(sentences))

loading rsm: 141it [00:00, 645.94it/s]


393698

## Ollama

In [23]:
import ollama
from evaluation import load_rsm

data = load_rsm()

In [26]:
text = data[0][0].input

response = ollama.chat(model='llama3.2:1b', messages=[ { 'role': 'user', 'content': f'summarize the text: <input text>{text}</input text> summarize the text within a sentence.', }, ])

In [27]:
print_in_lines(response['message']['content'], 100)

The text discusses various models and frameworks for managing data, including relational databases,
XML documents, and probabilistic data, with an emphasis on uncertainty in resource operation and
modeling non-1NF data. It highlights the limitations of existing models such as Non-First Normal
Form (Non-1NF) probabilistic models and proposes Probabilistic Resource Space Models to address
these issues by effectively specifying membership probability distributions of resources on
classification hierarchies.


In [30]:
from itertools import product

list(product([1, 2, 3], [5, 6, 7]))

[(1, 5), (1, 6), (1, 7), (2, 5), (2, 6), (2, 7), (3, 5), (3, 6), (3, 7)]

In [33]:
for model in ollama.ps()['models']:
    print(model.name)

llama3.1:latest


## PageRank


In [ ]:
import networkx as nx
import numpy as np

G = nx.DiGraph()
G.add_edge(1, 2, weight = 500)
G.add_edge(2, 3, weight = 300)
G.add_edge(3, 1, weight = 700)
G.add_edge(2, 4, weight = 2)
G.add_edge(4, 3, weight = 6)
G.add_edge('x', 'y', weight = 6)

# 计算带权重边的PageRank值
pagerank_dict = nx.pagerank(G, weight = 'weight', max_iter=1000, tol = 1e-6)
# 计算PageRank值
# pagerank_dict = nx.pagerank(G)

# 获取节点列表
nodes = list(G.nodes())
num_nodes = len(nodes)

# 初始化PageRank矩阵
pagerank_matrix = np.zeros((num_nodes, num_nodes))

# 填充PageRank矩阵
for i in range(num_nodes):
    from_node = nodes[i]
    out_edges = list(G.out_edges(from_node))
    out_degree = G.out_degree(from_node)
    for j in range(num_nodes):
        to_node = nodes[j]
        if (from_node, to_node) in out_edges:
            pagerank_matrix[i][j] = pagerank_dict[to_node] / out_degree


print(pagerank_matrix)

[[0.         0.27934012 0.         0.         0.         0.        ]
 [0.         0.         0.1499387  0.0177258  0.         0.        ]
 [0.28877526 0.         0.         0.         0.         0.        ]
 [0.         0.         0.29987741 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.06267645]
 [0.         0.         0.         0.         0.         0.        ]]


In [4]:
G.graph

{}